In [ ]:
import json

# JSON 파일 경로 지정
json_path = '../dataset/dev.json'  
output_path = '../dataset/dev_eng_trans.json'

# JSON 파일 읽기
with open(json_path, 'r', encoding='utf-8') as f:
    dataset = json.load(f)  # 리스트[딕셔너리] 형태 유지됨

print(len(dataset))

151


In [8]:
import re, os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.messages import HumanMessage

from dotenv import load_dotenv
load_dotenv()
#os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY"

True

In [14]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    request_timeout=60,
    api_key=os.environ["OPENAI_API_KEY"]
)

system_prompt = """
You are a Korean-to-English translation assistant.
You will receive a short conversation transcript written in Korean, consisting of three utterances.
Your task is to translate **only the last utterance** into natural English.
Respond with the translated sentence only — no extra text, comments, or formatting.
"""

integrated_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt.strip()),
    ("human", """
<Transcript>
{transcript}

<Translate the last utterance>
""")
])


In [16]:
from tqdm import tqdm
# 🔢 총 번역 문장 수 계산
total_utterances = sum(len(sample["input"]["conversation"]) for sample in dataset)

# 🔁 번역 처리
translated_dataset = []
progress = tqdm(total=total_utterances, desc="Translating utterances")

for sample in dataset:
    new_sample = sample.copy()
    new_sample["input"] = sample["input"].copy()
    new_sample["input"]["conversation"] = []

    conversation = sample["input"]["conversation"]

    for i in range(len(conversation)):
        start_idx = max(0, i - 2)
        context_slice = conversation[start_idx:i+1]

        if len(context_slice) == 0:
            continue

        # transcript 만들기
        transcript_lines = [
            f"speaker{utt['speaker']}: {utt['utterance']}" for utt in context_slice
        ]
        transcript = "\n".join(transcript_lines)

        # GPT 호출
        try:
            chain = integrated_prompt | llm
            response = chain.invoke({"transcript": transcript})
            translated = response.content.strip()
        except Exception as e:
            translated = f"[Error] {str(e)}"

        # 원 발화 복사 후 번역 추가
        new_utt = conversation[i].copy()
        new_utt["translated_utterance"] = translated
        new_sample["input"]["conversation"].append(new_utt)

        # ✅ tqdm 한 칸 증가
        progress.update(1)

    translated_dataset.append(new_sample)

progress.close()


Translating utterances: 100%|██████████| 3084/3084 [51:10<00:00,  1.00it/s]  


NameError: name 'output_path' is not defined

In [17]:
output_path = '../dataset/dev_english.json'
# 결과 저장
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(translated_dataset, f, ensure_ascii=False, indent=2)

print(f"✅ 저장 완료: {output_path}")

✅ 저장 완료: ../dataset/dev_english.json
